In [ ]:
import os
import numpy as np
import pandas as pd
import sys
import csv
import random
import tensorflow as tf
from keras import optimizers
from keras.callbacks import ModelCheckpoint, Callback
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, dot, multiply, Lambda
from keras.models import Model, Sequential, load_model
from keras.layers.wrappers import TimeDistributed
from keras.utils import to_categorical
from keras import backend as K
from random import shuffle
from sklearn.utils import class_weight
from datetime import datetime
random.seed(999)

In [ ]:
from data_generator_generalization import *

## Simulate generalization test with one-hot input

In [ ]:
for simulations in range(100):
    K.clear_session()
    print('simulation number: ', simulations+1)
    hiercorrect = 0
    absencecorrect = 0
    countabsent = 0
    
    ### generate data for generalization test: gen_type = 'int' or 'ext'
    input_train, output_train, input_test, output_test = gen_hier_onehot(train_size=500, gen_type='ext', random_training=False)

    # define class weights
    y_int = [y.argmax() for y in output_train]
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_int), y_int)

    ### if training is not random
    class_weights = {0: 0, 1: 0, 2: class_weights[0], 3: class_weights[1], 4: class_weights[2], 
                 5: class_weights[3], 6: class_weights[4], 7: class_weights[5], 8: class_weights[6]}

    ### if training is random
    '''
    class_weights = {0: 0, 1: 0, 2: 0, 3: class_weights[0], 4: class_weights[1], 5: class_weights[2], 
                 6: class_weights[3], 7: class_weights[4], 8: class_weights[5]}
    '''
    
    vec_in = Input(shape=(4, 74), dtype='float32', name='vec_in')
    lstm = LSTM(100, return_sequences=False, name='lstm1')(vec_in)
    target_pos = Dense(9, name='target_pos', activation='softmax')(lstm)
    model = Model(inputs=vec_in,outputs=target_pos)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(input_train,output_train, steps_per_epoch=100, epochs=50,verbose=1,class_weight=class_weights)
    
    eval_model = model.evaluate(x = input_test, y = output_test, verbose = 1) 
    loss = eval_model[0]
    accuracy = eval_model[1]
    
    predictions = model.predict(input_test)
    for testtrials in range(len(predictions)):
        index_prediction = list(predictions[testtrials]).index(max(predictions[testtrials]))+1
        index_answer = list(output_test[testtrials]).index(max(output_test[testtrials]))+1
        
        if index_answer == 9:
            countabsent += 1
            
        if index_prediction == index_answer:
            if index_answer == 9:
                absencecorrect += 1
            else:
                hiercorrect += 1
                
    abscor_percent = absencecorrect/countabsent*100
    hiercor_percent = hiercorrect/(100-countabsent)*100    
    
    if simulations == 0:
        evaluation = pd.DataFrame([[loss,accuracy,absencecorrect,abscor_percent,hiercorrect,hiercor_percent]], 
                                   columns=('loss', 'accuracy', 'absence', 'absence_perc', 'hierarchical', 'hierarchical_perc'))
    else:
        eval_model = pd.DataFrame([[loss,accuracy,absencecorrect,abscor_percent,hiercorrect,hiercor_percent]], 
                                  columns=('loss', 'accuracy', 'absence', 'absence_perc', 'hierarchical', 'hierarchical_perc'))
        evaluation = evaluation.append(eval_model, ignore_index = True)

    #evaluation.to_csv('data/generalization/extrapolation/h_gen1_sim_acc_500thirdred_onehot.csv', index=False)
    #evaluation.to_csv('data/generalization/interpolation/h_gen2_sim_acc_500thirdred_onehot.csv', index=False)


## Simulate generalization test with full word embeddings

In [ ]:
for simulations in range(100):
    K.clear_session()
    print('simulation number: ', simulations+1)
    hiercorrect = 0
    absencecorrect = 0
    countabsent = 0

    ### generate data for generalization test: gen_type = 'int' or 'ext'
    input_train, output_train, input_test, output_test = gen_hier_embful(train_size=500, gen_type='ext')

    # define class weights
    y_int = [y.argmax() for y in output_train]
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_int), y_int)
    class_weights = {0: 0, 1: 0, 2: class_weights[0], 3: class_weights[1], 4: class_weights[2], 
                 5: class_weights[3], 6: class_weights[4], 7: class_weights[5], 8: class_weights[6]}
    
    vec_in = Input(shape=(4, 364), dtype='float32', name='vec_in')
    lstm = LSTM(100, return_sequences=False, name='lstm1')(vec_in)
    target_pos = Dense(9, name='target_pos', activation='softmax')(lstm)
    model = Model(inputs=vec_in,outputs=target_pos)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(input_train,output_train, steps_per_epoch=100, epochs=50,verbose=1,class_weight=class_weights)
    
    eval_model = model.evaluate(x = input_test, y = output_test, verbose = 1) 
    loss = eval_model[0]
    accuracy = eval_model[1]
    
    predictions = model.predict(input_test)
    for testtrials in range(len(predictions)):
        index_prediction = list(predictions[testtrials]).index(max(predictions[testtrials]))+1
        index_answer = list(output_test[testtrials]).index(max(output_test[testtrials]))+1
        
        if index_answer == 9:
            countabsent += 1
            
        if index_prediction == index_answer:
            if index_answer == 9:
                absencecorrect += 1
            else:
                hiercorrect += 1
                
    abscor_percent = absencecorrect/countabsent*100
    hiercor_percent = hiercorrect/(100-countabsent)*100  
    
    if simulations == 0:
        evaluation = pd.DataFrame([[loss,accuracy,absencecorrect,abscor_percent,hiercorrect,hiercor_percent]], 
                                   columns=('loss', 'accuracy', 'absence', 'absence_perc', 'hierarchical', 'hierarchical_perc'))
    else:
        eval_model = pd.DataFrame([[loss,accuracy,absencecorrect,abscor_percent,hiercorrect,hiercor_percent]], 
                                  columns=('loss', 'accuracy', 'absence', 'absence_perc', 'hierarchical', 'hierarchical_perc'))
        evaluation = evaluation.append(eval_model, ignore_index = True)

    #evaluation.to_csv('data/generalization/extrapolation/h_gen1_sim_acc_500thirdred_embful.csv', index=False)
    #evaluation.to_csv('data/generalization/interpolation/h_gen2_sim_acc_500thirdred_embful.csv', index=False)


## Simulate generalization test with dimensionality-reduced word embeddings

In [ ]:
for simulations in range(100):
    K.clear_session()
    print('simulation number: ', simulations+1)
    hiercorrect = 0
    absencecorrect = 0
    countabsent = 0

    ### generate data for generalization test: gen_type = 'int' or 'ext'
    input_train, output_train, input_test, output_test = gen_hier_embred(train_size=500, gen_type='ext')
        
    # define class weights
    y_int = [y.argmax() for y in output_train]
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y_int), y_int)
    class_weights = {0: 0, 1: 0, 2: class_weights[0], 3: class_weights[1], 4: class_weights[2], 
                 5: class_weights[3], 6: class_weights[4], 7: class_weights[5], 8: class_weights[6]}
    
    vec_in = Input(shape=(4, 74), dtype='float32', name='vec_in')
    lstm = LSTM(100, return_sequences=False, name='lstm1')(vec_in)
    target_pos = Dense(9, name='target_pos', activation='softmax')(lstm)
    model = Model(inputs=vec_in,outputs=target_pos)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(input_train,output_train, steps_per_epoch=100, epochs=50,verbose=1,class_weight=class_weights)
    
    eval_model = model.evaluate(x = input_test, y = output_test, verbose = 1) 
    loss = eval_model[0]
    accuracy = eval_model[1]
    
    predictions = model.predict(input_test)
    for testtrials in range(len(predictions)):
        index_prediction = list(predictions[testtrials]).index(max(predictions[testtrials]))+1
        index_answer = list(output_test[testtrials]).index(max(output_test[testtrials]))+1
        
        if index_answer == 9:
            countabsent += 1
            
        if index_prediction == index_answer:
            if index_answer == 9:
                absencecorrect += 1
            else:
                hiercorrect += 1
                
    abscor_percent = absencecorrect/countabsent*100
    hiercor_percent = hiercorrect/(100-countabsent)*100 
    
    if simulations == 0:
        evaluation = pd.DataFrame([[loss,accuracy,absencecorrect,abscor_percent,hiercorrect,hiercor_percent]], 
                                   columns=('loss', 'accuracy', 'absence', 'absence_perc', 'hierarchical', 'hierarchical_perc'))
    else:
        eval_model = pd.DataFrame([[loss,accuracy,absencecorrect,abscor_percent,hiercorrect,hiercor_percent]], 
                                  columns=('loss', 'accuracy', 'absence', 'absence_perc', 'hierarchical', 'hierarchical_perc'))
        evaluation = evaluation.append(eval_model, ignore_index = True)

    #evaluation.to_csv('data/generalization/extrapolation/h_gen1_sim_acc_500thirdred_embred.csv', index=False)
    #evaluation.to_csv('data/generalization/interpolation/h_gen2_sim_acc_500thirdred_embred.csv', index=False)
